In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

votes_file = '../assets/datasets/votes.csv'
airport_file = '../assets/datasets/Airport_operations.csv'

# Case #1: Congressional Voting Data

After you've downloaded the data from the repository, go ahead and load it with Pandas

In [2]:
votes = pd.read_csv(votes_file)
votes.head()

,Unnamed: 0,Class,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,1,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,2,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,3,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,4,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,5,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


Next, let's define the x and y variables: 

In [3]:
votes = votes.ix[:, 1:]
votes.head(2)

,Class,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN


In [4]:
Y = [1 if x == 'democrat' else 0 for x in votes.Class.values]
print Y[0:10]

[0, 0, 1, 1, 1, 1, 1, 0, 0, 1]


In [5]:
Xvote = votes.iloc[:, 1:]
Xvote.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN


In [6]:
Xvote.isnull().sum()

V1      12
V2      48
V3      11
V4      11
V5      15
V6      11
V7      14
V8      15
V9      22
V10      7
V11     21
V12     31
V13     25
V14     17
V15     28
V16    104
dtype: int64

In [10]:
def change_yn_to_binary(x):
    if x == 'y':
        return 1
    elif x == 'n':
        return 0
    else:
        return x

# Because we would lose about half of the data if we dropped all null values, we're going to "impute" here.
# fill in the missing values with random 1 or 0:

# iterate through the columns
for c in Xvote.columns:
    Xvote.loc[:, c] = Xvote[c].map(change_yn_to_binary)
    # make a mask of rows for this column where the value is null/nan
    mask = Xvote[c].isnull()
    # for those rows at this column, put in random 1's or 0's the size of the number
    # of missing values
    Xvote.loc[mask, c] = np.random.choice(2, size = np.sum(mask))
    
Xvote.isnull().sum()

C:\Anaconda2\lib\site-packages\pandas\core\indexing.py:545: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Anaconda2\lib\site-packages\pandas\core\indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

V1     0
V2     0
V3     0
V4     0
V5     0
V6     0
V7     0
V8     0
V9     0
V10    0
V11    0
V12    0
V13    0
V14    0
V15    0
V16    0
dtype: int64

Next, create the covariance matrix from the standardized x-values and decompose these values to find the eigenvalues and eigenvectors

In [12]:
Xn = (Xvote - Xvote.mean()) / Xvote.std()
Xn_corr = Xn.cov()
Xn_corr

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
V1,1.000000,0.016977,0.389139,-0.415917,-0.348070,-0.398317,0.347050,0.396149,0.314194,-0.076609,0.115259,-0.407077,-0.327600,-0.339027,0.197382,0.108773
V2,0.016977,1.000000,-0.050680,0.064137,0.140163,0.143673,-0.221918,-0.105360,-0.153901,-0.121826,0.185436,-0.024548,0.232087,-0.012963,-0.103439,-0.064515
V3,0.389139,-0.050680,1.000000,-0.710194,-0.628838,-0.415098,0.560003,0.699074,0.580180,0.026278,0.214087,-0.650672,-0.501384,-0.555445,0.450612,0.275550
V4,-0.415917,0.064137,-0.710194,1.000000,0.741225,0.466395,-0.566233,-0.694566,-0.599136,0.034588,-0.286040,0.693898,0.576666,0.607677,-0.523519,-0.222664
V5,-0.348070,0.140163,-0.628838,0.741225,1.000000,0.615013,-0.664392,-0.797486,-0.728726,0.020718,-0.142262,0.631351,0.622964,0.664173,-0.543445,-0.241124
V6,-0.398317,0.143673,-0.415098,0.466395,0.615013,1.000000,-0.527397,-0.546542,-0.530139,0.108048,0.024578,0.511261,0.533697,0.562500,-0.422934,-0.192413
V7,0.347050,-0.221918,0.560003,-0.566233,-0.664392,-0.527397,1.000000,0.695036,0.615481,0.030572,0.019729,-0.515967,-0.546474,-0.488942,0.463212,0.275108
V8,0.396149,-0.105360,0.699074,-0.694566,-0.797486,-0.546542,0.695036,1.000000,0.702094,0.035119,0.136168,-0.606081,-0.596032,-0.603367,0.502572,0.277052
V9,0.314194,-0.153901,0.580180,-0.599136,-0.728726,-0.530139,0.615481,0.702094,1.000000,0.039035,0.038370,-0.563808,-0.489274,-0.516808,0.453615,0.243659
V10,-0.076609,-0.121826,0.026278,0.034588,0.020718,0.108048,0.030572,0.035119,0.039035,1.000000,-0.017316,0.048454,0.048366,0.110298,-0.076826,0.077751


Now, let's check the eigenvalues: 

In [13]:
eigenvalues, eigenvectors = np.linalg.eig(Xn_corr.values)

In [14]:
eigenvalues

array([ 7.15511309,  1.40472985,  1.1391514 ,  0.88206536,  0.84589317,
        0.77538254,  0.58480676,  0.53484941,  0.1588224 ,  0.50888715,
        0.22893064,  0.43598645,  0.35686353,  0.27888657,  0.40064942,
        0.30898226])

And the eigenvectors: 

In [15]:
print eigenvectors[0:2]
print len(eigenvectors[0])

[[ 0.18880909 -0.17076426  0.13099701 -0.26291937  0.63204762  0.49586205
   0.11690237  0.2966895   0.0756184  -0.15831857  0.032137   -0.04022251
   0.18613688  0.04296618  0.16355995 -0.07191144]
 [-0.0538208  -0.63365838 -0.0314694  -0.33037786  0.13470517 -0.54499682
   0.08954622 -0.07308253  0.02902259 -0.11081497 -0.11115146 -0.32258873
   0.07260681 -0.04049464 -0.12334431  0.07244407]]
16


To find the principal componants, find the eigenpairs, and sort them from highest to lowest. 

In [16]:
eigen_pairs = [[e_value, e_vector] for e_value, e_vector in zip(eigenvalues, eigenvectors)]
eigen_pairs = sorted(eigen_pairs, reverse = True)

In [17]:
[e[0] for e in eigen_pairs]

[7.1551130934316935,
 1.4047298506242887,
 1.1391513955138441,
 0.88206536457596962,
 0.8458931702976018,
 0.77538254305798182,
 0.58480676326112957,
 0.5348494073870782,
 0.50888714705214833,
 0.43598644925528179,
 0.40064942459911879,
 0.35686353026589585,
 0.30898226088250513,
 0.27888656590385513,
 0.22893063504260736,
 0.15882239884901572]

Now, calculate the explained variance. Recall the methods we learned in lesson 2.2!

In [18]:
total_variance = np.sum(eigenvalues)
explained_variance = [(e[0] / total_variance) * 100 for e in eigen_pairs]
explained_variance

[44.719456833948044,
 8.7795615664017959,
 7.1196962219615187,
 5.5129085285998052,
 5.286832314360006,
 4.8461408941123825,
 3.6550422703820562,
 3.3428087961692361,
 3.1805446690759243,
 2.7249153078455088,
 2.5040589037444905,
 2.2303970641618469,
 1.9311391305156553,
 1.7430410368990932,
 1.4308164690162948,
 0.9926399928063474]

Now, calculate the explained variance and the Cumulative explained variance

Now, conduct a PCA using scikit learn

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

# Case #2: Airport Delays

First, let's define the x and y variables: Airport is going to be our "x" variable

Then, standardize the x variable for analysis

Next, create the covariance matrix from the standardized x-values and decompose these values to find the eigenvalues and eigenvectors

Then, check your eigenvalues and eigenvectors:

To find the principal componants, find the eigenpairs, and sort them from highest to lowest. 

Next, Calculate the explained variance

Finally, conduct the PCA - use the results about to guide your selection of "n" componants